In [1]:
import numpy as np
import pandas as pd
from pandas import Series
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import scipy.stats as stats
import scipy.ndimage as ndimage
import pymannkendall as mk

In [2]:
# importing data
# numpy array with columns of date/time and num_value

PC_0192_details = pd.read_csv("Details_PC-0192.csv", sep=',')
# dtype='str', comments="#", delimiter="\t", unpack=False)
PC_0192_details

,Project ID,birth_deid_date,death_deid_date,sex,icu_visit,icu_start_deid_dttm,icu_end_deid_dttm,failed_extubation_flag,failed_extubation_deid_date,intubation_flag,extubation_flag,extubation_deid_date,extubation_score,re_intubation_deid_date,icu_ward
0,PC0192,22/06/2004,NaN,Male,1,22/12/2016 11:53,18/02/2017 17:13,1,10/01/2017 18:00,1,1,28/12/2016 18:00,5,11/01/2017 14:00,FLAMI


In [3]:
# changing required fields to correct datetime data type to avoid string issues

PC_0192_details['failed_extubation_deid_date'] = pd.to_datetime(PC_0192_details['failed_extubation_deid_date'], format='%d/%m/%Y %H:%M')
PC_0192_details['re_intubation_deid_date'] = pd.to_datetime(PC_0192_details['re_intubation_deid_date'], format='%d/%m/%Y %H:%M')

In [4]:
failed_extubation = PC_0192_details.iloc[0,8]
re_intubation = PC_0192_details.iloc[0,13]

print(failed_extubation, re_intubation)

2017-01-10 18:00:00 2017-01-11 14:00:00


In [9]:
PC_0192_RR_data = pd.read_csv("PC0192/PC0192_1_RR.csv", sep=',')
PC_0192_RR_data['record_date_time'] = pd.to_datetime(PC_0192_RR_data['record_date_time'], format='%Y-%m-%d %H:%M:%S')
PC_0192_RR_data


,monitor,record_date_time,num_value
0,RR,2016-12-22 12:09:31,17
1,RR,2016-12-22 12:09:36,18
2,RR,2016-12-22 12:09:41,17
3,RR,2016-12-22 12:09:46,17
4,RR,2016-12-22 12:09:51,17
...,...,...,...
1131421,RR,2017-02-18 17:08:34,36
1131422,RR,2017-02-18 17:08:39,35
1131423,RR,2017-02-18 17:08:44,31
1131424,RR,2017-02-18 17:08:49,30


In [24]:
# finding indexes at which failed_extubation and re_intubation would need to be inserted to maintain order

failed_extubation_index = PC_0192_RR_data.record_date_time.searchsorted(failed_extubation)
re_intubation_index = PC_0192_RR_data.record_date_time.searchsorted(re_intubation)

print(failed_extubation_index, re_intubation_index)

PC_0192_RR_extubated = PC_0192_RR_data.iloc[failed_extubation_index+1:re_intubation_index-1, :]
PC_0192_RR_extubated

436087 450068


,monitor,record_date_time,num_value
436088,combination,2017-01-10 18:00:06,28
436089,RR,2017-01-10 18:00:11,26
436090,RR,2017-01-10 18:00:16,26
436091,RR,2017-01-10 18:00:21,25
436092,RR,2017-01-10 18:00:26,29
...,...,...,...
450062,RR,2017-01-11 13:59:46,36
450063,awRR,2017-01-11 13:59:46,35
450064,RR,2017-01-11 13:59:52,37
450065,awRR,2017-01-11 13:59:52,37


In [22]:
# replacing duplicates with an average value
# only checking slices of the next 30 data points

for idx1, val1 in enumerate(PC_0192_RR_extubated['record_date_time']):
    for idx2, val2 in enumerate(PC_0192_RR_extubated['record_date_time'][idx1+1:idx1+50]):
        if val1 == val2:
            PC_0192_RR_extubated['num_value'].iloc[idx2] = 0.5*(PC_0192_RR_extubated['num_value'].iloc[idx1]+PC_0192_RR_extubated['num_value'].iloc[idx2])
            PC_0192_RR_extubated['monitor'].iloc[idx2] = "combination"
            PC_0192_RR_extubated = PC_0192_RR_extubated.drop(PC_0192_RR_extubated.index[idx1])


/Users/lucindakhalil/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/Users/lucindakhalil/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [25]:
PC_0192_RR_extubated.index

RangeIndex(start=436088, stop=450067, step=1)